In [25]:
import pandas as pd
import numpy as np
from typing import Dict, Any
from langchain_community.chat_models import ChatOllama
from alpha_vantage.timeseries import TimeSeries
import requests
from dotenv import load_dotenv 
import json

In [7]:
# Load environment variables
import os
load_dotenv() 
API_KEY = os.getenv('ALPHA_API_KEY')

In [21]:
import requests
class Brapi:
    def __init__(self):
        self.base_url = "https://brapi.dev/api/quote"
        
    def get_dalay(self, ticker, drange="1mo", interval="1d"):
        build_url = f"{self.base_url}/{ticker}?range={drange}&interval={interval}"
        response = requests.get(build_url)
        return response.json()

In [11]:
class LlamaAgent:
    def __init__(self, model="gemma3:1b", url="http://localhost:11434/api/generate"):
        self.model = model
        self.url = url

    def ask(self, prompt: str) -> str:
        resp = requests.post(self.url, json={"model": self.model, "prompt": prompt, "stream": False})
        return resp.json()["response"]


In [12]:
class MarketAgent:
    def __init__(self, api_key: str, cache_dir="data_cache", outputsize="compact"):
        self.ts = TimeSeries(key=api_key, output_format="pandas")
        self.cache_dir = cache_dir
        os.makedirs(cache_dir, exist_ok=True)
        self.outputsize = outputsize

    def run(self, ticker: str):
        cache_file = f"{self.cache_dir}/{ticker}.csv"
        try:
            # Tenta buscar dados da API
            df, meta = self.ts.get_daily(symbol=ticker, outputsize=self.outputsize)
            df = df.sort_index()
            df.to_csv(cache_file)  # salva no cache
            print(f"[INFO] Dados baixados online para {ticker}.")
        except Exception as e:
            print(f"[WARN] Falha ao baixar {ticker}: {e}")
            if os.path.exists(cache_file):
                df = pd.read_csv(cache_file, index_col=0, parse_dates=True)
                print(f"[INFO] Usando cache local: {cache_file}")
            else:
                raise RuntimeError(f"Sem dados disponíveis para {ticker}")

        # Calcula features
        df["ret"] = df["4. close"].pct_change()
        features = {
            "last_price": float(df["4. close"].iloc[-1]),
            "returns_7": float(df["ret"].tail(7).mean()),
            "vol_7": float(df["ret"].tail(7).std()),
            "sma_5": float(df["4. close"].tail(5).mean()),
            "sma_20": float(df["4. close"].tail(20).mean()),
        }
        return {"ticker": ticker, "features": features}


In [22]:
class MarketAgentBr:
    def __init__(self, cache_dir="data_cache"):
        self.cache_dir = cache_dir
        self.ts = Brapi()
        os.makedirs(cache_dir, exist_ok=True)

    def run(self, ticker: str):
        cache_file = f"{self.cache_dir}/{ticker}.json"
        try:
            data = self.ts.get_dalay(ticker)

            if "results" not in data:
                raise ValueError(f"Nenhum dado encontrado para {ticker}")

            # salva cache
            with open(cache_file, "w") as f:
                json.dump(data, f)

            print(f"[INFO] Dados baixados online para {ticker}.")
        except Exception as e:
            print(f"[WARN] Falha ao baixar {ticker}: {e}")
            if os.path.exists(cache_file):
                with open(cache_file, "r") as f:
                    data = json.load(f)
                print(f"[INFO] Usando cache local: {cache_file}")
            else:
                raise RuntimeError(f"Sem dados disponíveis para {ticker}")

        # transforma em dataframe para calcular indicadores
        prices = data["results"][0]["historicalDataPrice"]
        df = pd.DataFrame(prices)
        df["date"] = pd.to_datetime(df["date"], unit="s")
        df = df.sort_values("date")
        df["ret"] = df["close"].pct_change()

        # features
        features = {
            "last_price": float(df["close"].iloc[-1]),
            "returns_7": float(df["ret"].tail(7).mean()),
            "vol_7": float(df["ret"].tail(7).std()),
            "sma_5": float(df["close"].tail(5).mean()),
            "sma_20": float(df["close"].tail(20).mean()),
        }
        return {"ticker": ticker, "features": features}


In [13]:
class AnalystAgent:
    def __init__(self, llama_agent: LlamaAgent):
        self.llama = llama_agent

    def run(self, features: dict, horizon="7d", risk="medium"):
        prompt = f"""
        Você é um analista financeiro.
        Use os seguintes dados:
        Último preço: {features['last_price']:.2f}
        Retorno médio 7 dias: {features['returns_7']:.4f}
        Volatilidade 7 dias: {features['vol_7']:.4f}
        SMA-5: {features['sma_5']:.2f}
        SMA-20: {features['sma_20']:.2f}

        Contexto:
        - Horizonte de investimento: {horizon}
        - Perfil de risco: {risk}

        Tarefa:
        Diga se a recomendação é BUY, HOLD ou SELL.
        Explique em até 3 frases o motivo, incluindo os indicadores usados.
        """
        response = self.llama.ask(prompt)
        return {"analysis": response}


In [18]:
class CriticAgent:
    def __init__(self, llama_agent: LlamaAgent):
        self.llama = llama_agent

    def run(self, analysis: dict):
        prompt = f"""
        Você é um revisor de investimentos.
        Revise a seguinte recomendação de investimento:
        {analysis['analysis']}

        Tarefa:
        - Diga se a recomendação é coerente com os dados?
        - Diga se existe algum risco ou inconsistência?

        Responda de forma direta, curta, em bullet points.
        """
        review = self.llama.ask(prompt)
        return {"review": review}


In [15]:
class FinancialAdvisorSystem:
    def __init__(self, market_agent, analyst_agent, critic_agent):
        self.market_agent = market_agent
        self.analyst_agent = analyst_agent
        self.critic_agent = critic_agent

    def run(self, ticker: str, horizon="7d", risk="medium"):
        # 1. Dados de mercado
        market_result = self.market_agent.run(ticker)

        # 2. Análise
        analysis = self.analyst_agent.run(market_result["features"], horizon, risk)

        # 3. Crítica
        review = self.critic_agent.run(analysis)

        return {
            "ticker": ticker,
            "features": market_result["features"],
            "analysis": analysis,
            "review": review
        }


# 🔹 Tickers EUA (NASDAQ/NYSE)

| Empresa                   | Ticker                |
| ------------------------- | --------------------- |
| Apple Inc.                | **AAPL**              |
| Microsoft Corp.           | **MSFT**              |
| Alphabet (Google)         | **GOOG** ou **GOOGL** |
| Amazon.com Inc.           | **AMZN**              |
| Tesla Inc.                | **TSLA**              |
| Meta Platforms (Facebook) | **META**              |
| NVIDIA Corp.              | **NVDA**              |
| Netflix Inc.              | **NFLX**              |
| JPMorgan Chase & Co.      | **JPM**               |
| Bank of America Corp.     | **BAC**               |

---

# 🔹 Tickers Brasil (B3 – com sufixo `.SA`)

| Empresa            | Ticker       |
| ------------------ | ------------ |
| Petrobras PN       | **PETR4.SA** |
| Petrobras ON       | **PETR3.SA** |
| Vale ON            | **VALE3.SA** |
| Itaú Unibanco PN   | **ITUB4.SA** |
| Bradesco PN        | **BBDC4.SA** |
| Ambev ON           | **ABEV3.SA** |
| Magazine Luiza ON  | **MGLU3.SA** |
| Banco do Brasil ON | **BBAS3.SA** |
| Gerdau PN          | **GGBR4.SA** |
| Eletrobras ON      | **ELET3.SA** |

---

In [26]:
llama = LlamaAgent("gemma3:1b")
#market_agent = MarketAgent(API_KEY)
market_agent = MarketAgentBr()
analyst_agent = AnalystAgent(llama)
critic_agent = CriticAgent(llama)

advisor = FinancialAdvisorSystem(market_agent, analyst_agent, critic_agent)

result = advisor.run("PETR4", horizon="1mo", risk="medium")
print(result)


[INFO] Dados baixados online para PETR4.
{'ticker': 'PETR4', 'features': {'last_price': 30.59, 'returns_7': -0.00029477882817925086, 'vol_7': 0.00858111025422196, 'sma_5': 31.042, 'sma_20': 30.639999999999997}, 'analysis': {'analysis': 'Com base nos dados fornecidos, a recomendação é **HOLD**.\n\nA volatilidade de 7 dias é relativamente alta (0.0086), o que sugere um risco considerável. O retorno médio de 7 dias de -0.0003 indica que o investimento está gerando uma pequena perda, o que se alinha com o perfil de risco médio. O SMA-5 de 31.04 e SMA-20 de 30.64 mostram um movimento recente relativamente estável, indicando uma tendência de valorização ou desvalorização mínima. Considerando o horizonte de investimento de 1 mês e o perfil de risco de médio, manter o investimento em HOLD parece ser a opção mais prudente, pois o risco atual está dentro da tolerância média do investidor.'}, 'review': {'review': '* **A recomendação é coerente com os dados:** A combinação de alta volatilidade, re